In [ ]:
from glass.mob.esri import service_areas

facilities = '/mnt/disk1/dist_to_club/loc_clubs.shp'

breaks = ['5', '10', '15']

out = '/mnt/disk1/dist_to_club/sv_clubs_tv.shp'

impedance = 'TravelTime'

service_areas(facilities, breaks, out, impedance=impedance)

In [ ]:
from glass.mob.esri.cf import closest_facility

i   = '/mnt/d/rms/azores/rms_bgri21_pnt_smiguel.shp'
ipk = 'bgri'

f = '/mnt/d/rms/equipasaude/hsp_azores.shp'
out = '/mnt/d/rms/acc_pop21/azores_hsp.shp'

imp = 'TravelTime'

epsg = 5015

closest_facility(
    i, ipk, f, out, impedance=imp,
    crs=epsg, save_temp_json=True
)

In [ ]:
from glass.mob.esri.cf import cf_ign_distfac

i   = '/mnt/d/rms/azores/rms_bgri21_pnt_smiguel.shp'
ipk = 'bgri'

f = '/mnt/d/rms/lulc/evu_vtx_azores.shp'
out = '/mnt/d/rms/acc_pop21/azores_evu.shp'

epsg = 5015

cf_ign_distfac(
    i, ipk, f, out,
    crs=epsg, save_temp_json=True
)

In [ ]:
from glass.mob.esri.cf import cf_based_on_relations

i = '/mnt/d/rms/azores/rms_bgri21_pnt.shp'
ipk = 'bgri'
igroup = 'idfreg'

f = '/mnt/d/rms/equipasaude/cs_azores.shp'
fpk = 'csid'

reltbl = '/mnt/d/rms/equipasaude/AInf_CSP_Azores.xlsx'
sheet = 'A_Inf'
groupfk = 'ddccff'
ffk = 'idcs'

output = '/mnt/d/rms/acc_pop21/azores_cs.shp'

cf_based_on_relations(
    i, ipk, igroup,
    f, fpk, reltbl, sheet, groupfk, ffk,
    output, impedance='TravelTime'
)

In [ ]:
from glass.cons.esri import TV_URL, CF_URL, rest_token

from glass.rd.shp import shp_to_obj
from glass.prj.obj import df_prj
from glass.prop.prj import get_epsg
import requests as rq
from glass.cons.esri import get_tv_by_impedancetype
import json as js
from glass.it.esri    import json_to_gjson

In [ ]:
i   = '/mnt/d/rms/acc_pop21/tst_incidents.shp'
ipk = 'bgri'

f = '/mnt/d/rms/acc_pop21/tst_facilities.shp'
out = '/mnt/d/rms/acc_pop21/azores_idosos.shp'

In [ ]:
tv = get_tv_by_impedancetype("WalkTime")

In [ ]:
token = rest_token()

In [ ]:
tvs = rq.get(TV_URL, params={'f' : 'json', 'token' : token})

In [ ]:
tvs.json()

In [ ]:
epsg

In [ ]:
idf = shp_to_obj(i, srs_to=4326 if epsg != 4326 else None)
idf['lat'] = idf.geometry.y.astype(str)
idf['lng'] = idf.geometry.x.astype(str)

fdf = shp_to_obj(f, srs_to=4326 if epsg != 4326 else None)
fdf['lat'] = fdf.geometry.y.astype(str)
fdf['lng'] = fdf.geometry.x.astype(str)

idf["coords"] = idf.lng + ',' + idf.lat
fdf["coords"] = fdf.lng + ',' + fdf.lat

istr = idf.coords.str.cat(sep=";")
fstr = fdf.coords.str.cat(sep=";")

In [ ]:
fstr

In [ ]:
r = rq.get(CF_URL, params={
    #"impedanceAttributeName" : "TravelTime"
    'facilities'             : fstr,
    'incidents'              : istr,
    'token'                  : token,
    'f'                      : 'json',
    'travelModel'            : js.dumps(tv),
    'defaultTargetFacilityCount' : '1',
    'returnCFRoutes'         : True,
    'travelDirection'        : 'esriNATravelDirectionToFacility',
    'impedanceAttributeName' : 'WalkTime'
})

In [ ]:
r.status_code

In [ ]:
esri_geom = r.json()
geom = json_to_gjson(esri_geom.get('routes'))

In [ ]:
geom

In [ ]:
r.json()

In [ ]:
tv={
    "attributeParameterValues": [{
        "parameterName": "Restriction Usage",
        "attributeName": "Walking",
        "value": "PROHIBITED"
    },{
        "parameterName": "Restriction Usage",
        "attributeName": "Preferred for Pedestrians","value": "PREFER_LOW"},{"parameterName": "Walking Speed (km/h)","attributeName": "WalkTime","value": 5}],"description": "Follows paths and roads that allow pedestrian traffic and finds solutions that optimize travel time. The walking speed is set to 5 kilometers per hour.","impedanceAttributeName": "WalkTime","simplificationToleranceUnits": "esriMeters","uturnAtJunctions": "esriNFSBAllowBacktrack","restrictionAttributeNames": ["Preferred for Pedestrians","Walking"],"useHierarchy": false,"simplificationTolerance": 2,"timeAttributeName": "WalkTime","distanceAttributeName": "Miles","type": "WALK","id": "caFAgoThrvUpkFBW","name": "Walking Time"}

In [ ]:
tv